In [4]:
import numpy as np 
import h5py
import pickle

import util as yu
yu.flag_fast=False


infile='/capstor/store/cscs/userlab/s1174/lyan/code/projectData/NST_f/data/NST_f_cA2.09.48_Nsgm.h5'
# infile='/capstor/store/cscs/userlab/s1174/lyan/code/projectData/NST_f/data/NST_c_cA2.09.48.h5'
outfile='/capstor/store/cscs/userlab/s1174/lyan/code/projectData/NST_f/data_subtractionMethod/cA2.09.48.pkl'

data={}

with h5py.File(infile) as fr:
    # data['cfgs']=[cfg.decode() for cfg in fr[f'cfgs'][:]]
    print(fr[f'diags/N/opabs'][:])
    print(fr[f'diags/N/data/p_p'])
    data['twop_mom0']=np.real( yu.jackknife((fr[f'diags/N/data/p_p'][:,:,0]+fr[f'diags/N/data/p_p'][:,:,1])/2,d=2) )
    data['twop_mom1']=np.real( yu.jackknife((fr[f'diags/N/data/p_p'][:,:,2]+fr[f'diags/N/data/p_p'][:,:,3])/2,d=2) )
    
    print(fr['diags/NJN'].keys())
    print(fr['diags/NJN/inserts'][[5,6,7,8,9]])
    print(fr['diags/NJN/opabs'][[2,3,6,7]])
    
    # print(fr['diags/N-j/'].keys())

    data['G0_mom1']={}
    data['GA_mom1_momi0']={}
    data['GA_mom1_momi1']={}
    data['G5_mom1']={}
    for tf in [10,12,14]:
        t=yu.jackknife(fr[f'diags/NJN/data/p_j-_p_deltat_{tf}'],d=2)
        
        data['G0_mom1'][f'dt{tf}']=np.real( (t[:,:,2,9]-t[:,:,7,9]) ) 
        data['GA_mom1_momi0'][f'dt{tf}']=np.real( 1j*(t[:,:,3,6]+t[:,:,6,6]) -  (t[:,:,3,7]-t[:,:,6,7]) )/2
        data['GA_mom1_momi1'][f'dt{tf}']=np.real( 1j*(t[:,:,2,8]-t[:,:,7,8]) )
        data['G5_mom1'][f'dt{tf}']=np.real( (t[:,:,2,5]-t[:,:,7,5]) ) /2 # the 1/2 is meant to match Simone's file
        
        
    data['G0_mom1_disc']={}
    data['GA_mom1_momi0_disc']={}
    data['GA_mom1_momi1_disc']={}
    data['G5_mom1_disc']={}
    for tf in [10,12,14]:
        t=yu.jackknife(fr[f'diags/N-j/data/p_j-_p_deltat_{tf}'],d=2)
        
        data['G0_mom1_disc'][f'dt{tf}']=np.real( (t[:,:,2,9]-t[:,:,7,9]) ) 
        data['GA_mom1_momi0_disc'][f'dt{tf}']=np.real( 1j*(t[:,:,3,6]+t[:,:,6,6]) )
        data['GA_mom1_momi1_disc'][f'dt{tf}']=np.real( 1j*(t[:,:,2,8]-t[:,:,7,8]) )
        data['G5_mom1_disc'][f'dt{tf}']=np.real( (t[:,:,2,5]-t[:,:,7,5]) ) /2 # the 1/2 is meant to match Simone's file

with open(outfile,'wb') as f:
    pickle.dump(data,f) 
    
    

[b'g;0,0,0;G1g;a;l1;N_g;0,0,0;G1g;a;l1;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,0;G1g;a;l2;N'
 b'g;0,0,1;G1;a;l1;N_g;0,0,1;G1;a;l1;N'
 b'g;0,0,1;G1;a;l2;N_g;0,0,1;G1;a;l2;N']
<HDF5 dataset "p_p": shape (1228, 24, 4), type "<c8">
<KeysViewHDF5 ['data', 'inserts', 'opabs', 'srcs']>
[b'g5' b'g5gx' b'g5gy' b'g5gz' b'g5gt']
[b'g;0,0,0;G1g;a;l1;N_g;0,0,1;G1;a;l1;N'
 b'g;0,0,0;G1g;a;l1;N_g;0,0,1;G1;a;l2;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,1;G1;a;l1;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,1;G1;a;l2;N']


In [155]:
# combine disc with Simone's files

enss=['b','c','d']

def invjk(dat):
    t=np.sum(dat,axis=0)
    return t[None,...] - dat * (len(dat)-1)

for ens in enss:
    t1={'b':'B72.64_6','c':'C60.80_6','d':'D54.96_6'}[ens]
    inpath_conn=f'/capstor/store/cscs/userlab/s1174/lyan/code/projectData/NST_f/data_subtractionMethod/old/{t1}.pkl'
    t2={'b':'B64','c':'C80','d':'D96'}[ens]
    inpath_disc=f'/capstor/store/cscs/userlab/s1174/lyan/code/projectData/fromBooster/projectDataMigrating/NST_b-discNJN/data/NST_b.h5_c{t2}'
    outpath=f'/capstor/store/cscs/userlab/s1174/lyan/code/projectData/NST_f/data_subtractionMethod/{t1}.pkl'
    data=pickle.load(open(inpath_conn,'rb'))
    cfgs_conn=list(data['confs'])
    data['confs']=cfgs_conn
    # cfgs_conn.sort()
    if ens=='b':
        cfgs_conn=[{'0':'a','1':'b'}[cfg[0]]+cfg[-4:] for cfg in cfgs_conn]
        cfgs_conn.sort()
    else:
        cfgs_conn=[{'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]]+cfg[:4] for cfg in cfgs_conn]
    # print(cfgs_conn)
    # print(data.keys())
    
    with h5py.File(inpath_disc) as fr:
        cfgs_disc=[cfg.decode() for cfg in fr['cfgs'][:]]
        cfgs=list(set(cfgs_conn).intersection(set(cfgs_disc)))
        cfgs.sort()

        print(ens,len(cfgs),len(cfgs_conn),len(cfgs_disc))
        
        dic={}
        for i,cfg in enumerate(cfgs_conn):
            dic[cfg]=i
        map_conn=[dic[cfg] for cfg in cfgs]
        for key in data.keys():
            t=data[key]
            if type(t) in [np.ndarray]:
                t=invjk(data[key])[map_conn]
                data[key]=yu.jackknife(t)
            elif type(t) in [dict]:
                for key2 in t.keys():
                    t=invjk(data[key][key2])[map_conn]
                    data[key][key2]=yu.jackknife(t)
            elif type(t) in [list]:
                data[key]=list(np.array(data[key])[map_conn])
            else:
                1/0
                
        dic={}
        for i,cfg in enumerate(cfgs_disc):
            dic[cfg]=i
        map_disc=[dic[cfg] for cfg in cfgs]
        tfs=[int(tf[2:]) for tf in data['G0_mom1'].keys()]
        tfs.sort()
        data['G0_mom1_disc']={}
        data['GA_mom1_momi0_disc']={}
        data['GA_mom1_momi1_disc']={}
        data['G5_mom1_disc']={}
        print(fr['diags/N-j'].keys())
        print(fr['diags/N-j/inserts'][[5,6,7,8,9]])
        print(fr['diags/N-j/opabs'][[2,3,6,7]])
        for tf in tfs:
            t=fr[f'diags/N-j/data/N2_j-_N2_deltat_{tf}'][:]
            t=t[map_disc]
            t=yu.jackknife(t,d=1)
            
            # Here is an additional -1 sign to match with the A ensemble result, but I don't know why (likely to be different use of N1/N2)
            data['G0_mom1_disc'][f'dt{tf}']=np.real( (t[:,:,2,9]-t[:,:,7,9]) ) * (-1)
            data['GA_mom1_momi0_disc'][f'dt{tf}']=np.real( 1j*(t[:,:,3,6]+t[:,:,6,6]) ) * (-1)
            data['GA_mom1_momi1_disc'][f'dt{tf}']=np.real( 1j*(t[:,:,2,8]-t[:,:,7,8]) ) * (-1)
            data['G5_mom1_disc'][f'dt{tf}']=np.real( (t[:,:,2,5]-t[:,:,7,5]) ) /2 * (-1) # the 1/2 is meant to match Simone's file
    
    with open(outpath,'wb') as f:
        pickle.dump(data,f) 


b 743 750 743
<KeysViewHDF5 ['data', 'inserts', 'opabs', 'srcs']>
[b'g5' b'g5gx' b'g5gy' b'g5gz' b'g5gt']
[b'g;0,0,0;G1g;a;l1;N_g;0,0,1;G1;a;l1;N'
 b'g;0,0,0;G1g;a;l1;N_g;0,0,1;G1;a;l2;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,1;G1;a;l1;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,1;G1;a;l2;N']
c 400 401 400
<KeysViewHDF5 ['data', 'inserts', 'opabs', 'srcs']>
[b'g5' b'g5gx' b'g5gy' b'g5gz' b'g5gt']
[b'g;0,0,0;G1g;a;l1;N_g;0,0,1;G1;a;l1;N'
 b'g;0,0,0;G1g;a;l1;N_g;0,0,1;G1;a;l2;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,1;G1;a;l1;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,1;G1;a;l2;N']
d 494 496 494
<KeysViewHDF5 ['data', 'inserts', 'opabs', 'srcs']>
[b'g5' b'g5gx' b'g5gy' b'g5gz' b'g5gt']
[b'g;0,0,0;G1g;a;l1;N_g;0,0,1;G1;a;l1;N'
 b'g;0,0,0;G1g;a;l1;N_g;0,0,1;G1;a;l2;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,1;G1;a;l1;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,1;G1;a;l2;N']


In [143]:
# combine disc with Simone's files 
# here I try to reproduce Simone's cfg order based on 2pt
# I can only do it for C, since B and D I use less sources.

enss=['b','c','d']

def invjk(dat):
    t=np.sum(dat,axis=0)
    return t[None,...] - dat * (len(dat)-1)

for ens in enss[1:]:
    t1={'b':'B72.64_6','c':'C60.80_6','d':'D54.96_6'}[ens]
    inpath_conn=f'/capstor/store/cscs/userlab/s1174/lyan/code/projectData/NST_f/data_subtractionMethod/old/{t1}.pkl'
    t2={'b':'B64','c':'C80','d':'D96'}[ens]
    inpath_disc=f'/capstor/store/cscs/userlab/s1174/lyan/code/projectData/fromBooster/projectDataMigrating/NST_b-discNJN/data/NST_b.h5_c{t2}'
    outpath=f'/capstor/store/cscs/userlab/s1174/lyan/code/projectData/NST_f/data_subtractionMethod/{t1}.pkl'
    data=pickle.load(open(inpath_conn,'rb'))
    cfgs_conn=list(data['confs'])
    data['confs']=cfgs_conn
    # cfgs_conn.sort()
    if ens=='b':
        cfgs_conn=[{'0':'a','1':'b'}[cfg[0]]+cfg[-4:] for cfg in cfgs_conn]
        # cfgs_conn.sort()
    else:
        cfgs_conn=[{'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]]+cfg[:4] for cfg in cfgs_conn]
    print(cfgs_conn)
    print(data.keys())
    
    with h5py.File(inpath_disc) as fr:
        print(fr.keys())
        cfgs_disc=[cfg.decode() for cfg in fr['cfgs'][:]]
        cfgs=list(set(cfgs_conn).intersection(set(cfgs_disc)))
        cfgs.sort()

        print(len(cfgs),len(cfgs_conn),len(cfgs_disc))
        
        # dic={}
        # for i,cfg in enumerate(cfgs_conn):
        #     dic[cfg]=i
        # map_conn=[dic[cfg] for cfg in cfgs]
        # for key in data.keys():
        #     t=data[key]
        #     if type(t) in [np.ndarray]:
        #         t=invjk(data[key])[map_conn]
        #         data[key]=yu.jackknife(t)
            # elif type(t) in [dict]:
            #     for key2 in t.keys():
            #         t=invjk(data[key][key2])[map_conn]
            #         data[key][key2]=yu.jackknife(t)
            # elif type(t) in [list]:
            #     data[key]=list(np.array(data[key])[map_conn])
            # else:
            #     1/0
                

        dic={}
        for i,cfg in enumerate(cfgs_disc):
            dic[cfg]=i
        map_disc=[dic[cfg] for cfg in cfgs]

        t1=invjk(data['twop_mom0'])
        # print(fr[f'diags/N/opabs'][:])
        t2=(fr[f'diags/N/data/N2_N2'][:,:,0]+fr[f'diags/N/data/N2_N2'][:,:,1])/2
        t2=np.real(t2[map_disc])
        # t2=np.real(yu.jackknife(t,d=1))

        ifortest=10
        print(t1[:4,ifortest])
        print(t2[:4,ifortest])

        t1for2=t1[:,ifortest]
        map_conn_rec=[]
        cfgs_conn_rec=[None for i in t1for2]
        for ele in t2[:,ifortest]:
            tmp=np.abs(t1for2-ele)
            ind=int(np.argmin(tmp))
            map_conn_rec.append(ind)
        print(len(map_conn_rec),len(set(map_conn_rec)))
        print(map_conn_rec)
        for i,ele in enumerate(map_conn_rec):
            cfgs_conn_rec[ele]=cfgs[i]
        print(cfgs_conn_rec)
        
        print(t1[map_conn_rec][:4,ifortest])
        print(t2[:4,ifortest])
    
    with open(outpath,'wb') as f:
        pickle.dump(data,f) 
    break

['a1204', 'b0264', 'b1208', 'a2028', 'a1076', 'b0856', 'a1972', 'b0088', 'a1692', 'b1224', 'a2148', 'b0296', 'a1980', 'b0120', 'a1892', 'b0248', 'b0488', 'a0644', 'a0788', 'b0168', 'b0200', 'a1796', 'a0668', 'a1020', 'a1324', 'a1916', 'b1248', 'a2044', 'b1112', 'b0104', 'a1052', 'a1700', 'b0752', 'a1812', 'b0832', 'b0864', 'a0772', 'b0704', 'a0956', 'a2180', 'a1748', 'b0480', 'a0612', 'b0208', 'b0352', 'a1468', 'a1516', 'a2140', 'b1000', 'a2116', 'b0560', 'b0952', 'a1252', 'b0536', 'a1508', 'a2060', 'b0312', 'b0728', 'b0376', 'a2068', 'b0336', 'a0708', 'a1668', 'a1820', 'b0448', 'b1120', 'a0508', 'b1024', 'b1168', 'b0464', 'a1476', 'b1064', 'a1532', 'b0688', 'b1376', 'a0828', 'a1644', 'b1104', 'a0556', 'b0472', 'a2100', 'a0724', 'b1216', 'a1180', 'b0152', 'b0816', 'b0912', 'b0008', 'a0532', 'a1420', 'b1152', 'a0884', 'b0192', 'b0144', 'b0288', 'b0360', 'a1348', 'b0656', 'b0456', 'b0616', 'a1428', 'b1280', 'a1340', 'b1464', 'a1764', 'b1416', 'b0608', 'a1828', 'a1332', 'a1572', 'a0628', 

In [7]:
import numpy as np 
import h5py
import pickle

import util as yu
yu.flag_fast=False

enss=['b','c','d']
for ens in enss:
    t={'a':'cA2.09.48','b':'B72.64_6','c':'C60.80_6','d':'D54.96_6'}[ens]
    path=f'/capstor/store/cscs/userlab/s1174/lyan/code/projectData/NST_f/data_subtractionMethod/{t}.pkl'
    t=pickle.load(open(path,'rb'))
    
    cfgs=t['confs']
    
    
    print(len(cfgs))

750
401
496
